# Ukázka implementace NN od doktora Trnečky

In [1]:
import random
import math

# jednodušší data pro lepší představu
zero_digit = [1,1,1,1,1,
              1,0,0,0,1,
              1,0,0,0,1,
              1,0,0,0,1,
              1,1,1,1,1]

one_digit = [0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0]

two_digit = [1,1,1,1,1,
             0,0,0,0,1,
             1,1,1,1,1,
             1,0,0,0,0,
             1,1,1,1,1]

three_digit = [1,1,1,1,1,
               0,0,0,0,1,
               1,1,1,1,1,
               0,0,0,0,1,
               1,1,1,1,1]

four_digit = [1,0,0,0,1,
              1,0,0,0,1,
              1,1,1,1,1,
              0,0,0,0,1,
              0,0,0,0,1]

five_digit = [1,1,1,1,1,
              1,0,0,0,0,
              1,1,1,1,1,
              0,0,0,0,1,
              1,1,1,1,1]

six_digit = [1,1,1,1,1,
             1,0,0,0,0,
             1,1,1,1,1,
             1,0,0,0,1,
             1,1,1,1,1]

seven_digit = [1,1,1,1,1,
               0,0,0,0,1,
               0,0,0,0,1,
               0,0,0,0,1,
               0,0,0,0,1]

eight_digit = [1,1,1,1,1,
               1,0,0,0,1,
               1,1,1,1,1,
               1,0,0,0,1,
               1,1,1,1,1]

nine_digit = [1,1,1,1,1,
              1,0,0,0,1,
              1,1,1,1,1,
              0,0,0,0,1,
              1,1,1,1,1]

def print_digit(digit):
    for i in range(0,25,5):
        print(''.join(['@' if i==1 else '.' for i in digit[i:i+5]]))

imputAll = [zero_digit, two_digit, three_digit, four_digit, five_digit, six_digit, seven_digit, eight_digit, nine_digit]
        
for digit in imputAll:
    print_digit(digit)
    print()

@@@@@
@...@
@...@
@...@
@@@@@

@@@@@
....@
@@@@@
@....
@@@@@

@@@@@
....@
@@@@@
....@
@@@@@

@...@
@...@
@@@@@
....@
....@

@@@@@
@....
@@@@@
....@
@@@@@

@@@@@
@....
@@@@@
@...@
@@@@@

@@@@@
....@
....@
....@
....@

@@@@@
@...@
@@@@@
@...@
@@@@@

@@@@@
@...@
@@@@@
....@
@@@@@



In [2]:
# implementace jednoduché neuronové sítě se třemi vrstvami
# pro jednoduchost vynecháme bias

import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class Simple_NN:
    # inicializace, nastavíme váhy na náhodné hodnoty, 0.01 kvůli rychlejší konvergenci
    def __init__(self, input_size, hidden_size, output_size):
        self.input_hidden_w = np.random.randn(input_size, hidden_size)
        self.hidden_output_w = np.random.randn(hidden_size, output_size)

        
    # průchod neuronovou sítí
    def forward(self, inputs):
        # přechod ze vstupní vrstvy do skryté
        self.hidden_output = sigmoid(np.dot(inputs, self.input_hidden_w))
        
        # přechod ze skryté vrstvy do výstupní vrstvy
        self.final_output = sigmoid(np.dot(self.hidden_output, self.hidden_output_w))
        
        return self.final_output

    
    # backpropagation
    def backward(self, inputs, target, learning_rate):
        self.forward(inputs)
        
        loss = target - self.final_output # triviální, existují jiné míry   
        output_delta = loss * sigmoid_derivative(self.final_output) # rozdíl
        self.hidden_output_w += self.hidden_output.T.dot(output_delta) * learning_rate
        
        hidden_loss = output_delta.dot(self.hidden_output_w.T)
        hidden_delta = hidden_loss * sigmoid_derivative(self.hidden_output)
        self.input_hidden_w += inputs.T.dot(hidden_delta) * learning_rate
           
            
    # trénování
    def train(self, inputs, targets, learning_rate, epochs):
        for _ in range(epochs):
            for i in range(len(inputs)):
                # aaaa!
                input_data = np.expand_dims(inputs[i], axis=0)
                target_data = np.expand_dims(targets[i], axis=0)

                self.backward(input_data, target_data, learning_rate)
                
            if (_ % 1000 == 0):
                print("Error:", np.mean(np.square(targets - self.forward(inputs))))

In [3]:
# jednoduchý příklad, naučíme naši síť funkci XOR
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
targets = np.array([[0], [1], [1], [0]])

nn = Simple_NN(2, 4, 1)
nn.train(inputs, targets, learning_rate=0.5, epochs=5000)

# test
for i in range(len(inputs)):
    input_data = inputs[i]
    predicted_output = nn.forward(input_data)
    print("in:", input_data, "predicted:", predicted_output)

Error: 0.3377986581118194
Error: 0.025687391357638735
Error: 0.0061572963819655786
Error: 0.003373832488235069
Error: 0.0023023160056748616
in: [0 0] predicted: [0.062802]
in: [0 1] predicted: [0.9629516]
in: [1 0] predicted: [0.96286476]
in: [1 1] predicted: [0.01636096]


In [4]:
# zkusíme naučit rozpoznávání číslic
inputs = np.array([zero_digit, one_digit, two_digit, three_digit, four_digit, 
          five_digit, six_digit, seven_digit, eight_digit, nine_digit])

targets = np.array([[1 if i==j else 0 for i in range(10)] for j in range(10)])

nn = Simple_NN(25, 16, 10)
nn.train(inputs, targets, learning_rate=0.1, epochs=20000)

Error: 0.23731156594310676
Error: 0.01976332992951258
Error: 0.00966097386435705
Error: 0.0013344952777337626
Error: 0.0006672490131822154
Error: 0.00044906430919571525
Error: 0.00033907201202729463
Error: 0.0002723517473529495
Error: 0.00022745081613569012
Error: 0.00019513903149198395
Error: 0.00017076575052994426
Error: 0.00015172563843668038
Error: 0.00013644304487622105
Error: 0.00012390806032952452
Error: 0.00011344311845944149
Error: 0.00010457651316132299
Error: 9.696966868877777e-05
Error: 9.037324228761591e-05
Error: 8.459953654293594e-05
Error: 7.950455750949617e-05


In [5]:
test_1 = [0,1,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,1,0,1,1,1,0]
print_digit(test_1)
result = nn.forward(test_1)
print(["%0.3f" % i for i in result])

.@@@.
...@@
..@@.
...@@
.@@@.
['0.002', '0.171', '0.006', '0.130', '0.000', '0.000', '0.000', '0.011', '0.046', '0.279']


In [6]:
test_2 = [1,0,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,0,1,1]
print_digit(test_2)
result = nn.forward(test_2)
print(["%0.3f" % i for i in result])

@.@@@
.....
.@@@@
....@
@@.@@
['0.000', '0.002', '0.003', '0.075', '0.020', '0.577', '0.001', '0.118', '0.000', '0.001']
